In [34]:

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import balanced_accuracy_score

In [35]:
X_train = np.load("x_train.npy")
y_train = np.load("y_train.npy")
#X_train = np.load("x_smote.npy")
#y_train = np.load("y_smote.npy")
x_project = np.load("Xtest_Classification1.npy")
X_test = np.load("x_test.npy")
y_test = np.load("y_test.npy")

print(X_train.shape,y_train.shape,x_project.shape,X_test.shape,y_test.shape)

(6449, 28, 28, 3) (6449, 2) (1764, 2352) (1251, 28, 28, 3) (1251, 2)


In [36]:
X_train = np.pad(X_train, ((0, 0), (2, 2), (2, 2), (0, 0)), mode='constant', constant_values=0)
X_test = np.pad(X_test, ((0, 0), (2, 2), (2, 2), (0, 0)), mode='constant', constant_values=0)

In [41]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

for layer in base_model.layers:
    layer.trainable = False

x = Flatten()(base_model.output)
x = Dense(256, activation='relu')(x)
x = Dense(128, activation='relu')(x)
output = Dense(2, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

model.compile(optimizer=Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose = 3, restore_best_weights=True)

history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test) ,callbacks=[early_stopping])

train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

plt.figure(figsize=(10, 6))
plt.plot(range(1, len(train_accuracy) + 1), train_accuracy, label='Training Accuracy')
plt.plot(range(1, len(val_accuracy) + 1), val_accuracy, label='Validation Accuracy')

test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_accuracy}')



Epoch 1/10
101/101 [==============================] - 24s 225ms/step - loss: 0.5335 - accuracy: 0.7246 - val_loss: 0.4250 - val_accuracy: 0.8090
Epoch 2/10
 99/101 [============================>.] - ETA: 0s - loss: 0.4647 - accuracy: 0.7702

KeyboardInterrupt: 

In [ ]:
y_predi = model.predict(X_test)

for i in y_predi:
    if i[0]>i[1]:
        i[0]=1
        i[1]=0
    else:
        i[0]=0
        i[1]=1
        
print(y_predi[:,0])
balanced_accuracy = balanced_accuracy_score(y_test, y_predi)
print("Balanced Accuracy Score:", balanced_accuracy)



ValueError: Input size must be at least 32x32; Received: input_shape=(28, 28, 3)

In [ ]:
model.summary()

In [ ]:
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay
cm = confusion_matrix(y_predi[:,0],y_test[:,0])
cm_display=ConfusionMatrixDisplay(cm).plot()